**UPDATE:** Seems that Kaggle kernel doesn't support **bokeh** visualization. So, I've uploaded the ipython notebook at [github][1].

In the competition, [New York City Taxi Trip Duration][2], Kaggle is challenging you to build a model that predicts the total ride duration of taxi trips in New York City. Your primary dataset is one released by the NYC Taxi and Limousine Commission, which includes pickup time, geo-coordinates, number of passengers, and several other variables.
In this notebook, we'll try to make some visualization using New York City Taxi Trip Duration data. There are many good kernels and discussions related to EDA of NYC Taxi data, however, we'll demonstrate how the datashader and bokeh library help make large dataset truely practical.

[Datashader][3] is a data rasterization pipeline for automating the process of creating meaningful representations of large amounts of data. Datashader breaks the creation of images of data into 3 main steps: Projection, Aggregation and Transformation. These aggregates are then further processed, eventually creating an image. Using this very general pipeline, many interesting data visualizations can be created in a performant and scalable way. Datashader contains tools for easily creating these pipelines in a composable manner, using only a few lines of code. Datashader can be used on its own, but it is also designed to work as a pre-processing stage in a plotting library, allowing that library to work with much larger datasets than it would otherwise.

[Bokeh][4] is a Python interactive visualization library that targets modern web browsers for presentation. Its goal is to provide elegant, concise construction of novel graphics in the style of D3.js, and to extend this capability with high-performance interactivity over very large or streaming datasets. Bokeh can help anyone who would like to quickly and easily create interactive plots, dashboards, and data applications.


  [1]: https://github.com/akhanss/nyc-taxi-trip-duration/blob/master/Interactive%20Data%20Visualization%20using%20datashader%20and%20bokeh.ipynb
  [2]: https://www.kaggle.com/c/nyc-taxi-trip-duration
  [3]: https://github.com/bokeh/datashader
  [4]: http://bokeh.pydata.org/en/latest/

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

**Load NYC Taxi Trip Data**

In [ ]:
%time
# We'll load some important columns only
df = pd.read_csv('../input/train.csv',
                 usecols=['pickup_datetime', 'dropoff_datetime', 'passenger_count', 'pickup_longitude', 'pickup_latitude',
                          'dropoff_longitude', 'dropoff_latitude',  'trip_duration'])

In [ ]:
# Let's see some records
df.head()

The training set contains about 1.5 million pickup and dropoff locations with passenger counts and trip durations.

In [ ]:
# size of the training set
print('Size:', df.shape[0])

In [ ]:
from bokeh.plotting import figure, output_notebook, show # bokeh plotting library
# We'll show the plots in the cells of this notebook
output_notebook()

In [ ]:
import numpy as np # linear algebra and Scientific calculation

# let's find pickup and dropoff longitude and latitude range
print(np.min(df['pickup_longitude']), np.min(df['pickup_latitude']))
print(np.max(df['pickup_longitude']), np.max(df['pickup_latitude']))

print(np.min(df['dropoff_longitude']), np.min(df['dropoff_latitude']))
print(np.max(df['dropoff_longitude']), np.max(df['dropoff_latitude']))

So, we've x_range (-121.93334198, -61.3355293274) and y_range is (32.1811408997, 51.8810844421). However, most of the rides occur within the x_range (-74.05, -73.7) and y_range (40.6, 40.9). For nice visualation, we'll omit the outliers from the initial plot. But you can explore the data using the **wheel zooming** facility of the bokeh plot.

**Let's define a base plot**

In [ ]:
# NYC = x_range, y_range = ((-121.93334198, -61.3355293274), (32.1811408997, 51.8810844421))
NYC = x_range, y_range = ((-74.05, -73.7), (40.6, 40.9))

plot_width = int(750)
plot_height = int(plot_width//1.2)

def base_plot(tools='pan, wheel_zoom, reset', plot_width=plot_width, plot_height=plot_height, **plot_args):
    p = figure(tools=tools, plot_width=plot_width, plot_height=plot_height,
              x_range=x_range, y_range=y_range, outline_line_color=None,
              min_border=0, min_border_left=0, min_border_right=0,
              min_border_top=0, min_border_bottom=0, **plot_args)
    
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    return p

options = dict(line_color=None, fill_color='blue', size=5)

**Let's make 10K pickup-points scatterplot**

In [ ]:
%%time
# let's plot 10k sample pickup
samples = df.sample(n=10000)
p = base_plot()

p.circle(x=samples['pickup_longitude'], y=samples['pickup_latitude'], **options)
show(p)

This size of the plot can be handled by any plotting library. Here the points are initially overplotting each other, but using zooming button, nearly all of them should be ** clearly visible in the bokeh plot **.

**Here is 10K dropoff-points scatter plot**

In [ ]:
%%time
# Again, let's plot 10k sample dropoff
samples = df.sample(n=10000)
p = base_plot()

p.circle(x=samples['dropoff_longitude'], y=samples['dropoff_latitude'], **options)
show(p)

We've plotted a subsample of data in the above figures and those are not so dense. When datasize grows to include millions and billions of points, traditional visualization techniques break down. Whether you're loading the data into limited memory, or separating the signal from the noise when thousands of data points occupy each pixel, as data gets big, visualization gets challenging.

**Datashader** deconstructs the classical visualization pipeline to place statistical processing at the heart of the visualization task. The result is a scalable, interactive system that is easy to use and produces perceptually accurate renderings of extremely large datasets.

In [ ]:
import datashader as ds
from datashader import transfer_functions as tr_fns
from datashader.colors import Greys9
Greys9_r = list(reversed(Greys9))[:2]

In [ ]:
%%time
cvs = ds.Canvas(plot_width=plot_width, plot_height=plot_height, x_range=x_range, y_range=y_range)
agg = cvs.points(df, 'dropoff_longitude', 'dropoff_latitude', ds.count('passenger_count'))
img = tr_fns.shade(agg, cmap=["white", 'darkblue'], how='linear')

img

In [ ]:
from datashader.bokeh_ext import InteractiveImage
from functools import partial
from datashader.utils import export_image
from datashader.colors import colormap_select, Greys9, Hot, viridis, inferno
from IPython.core.display import HTML, display

background = "black"
export = partial(export_image, export_path="export", background=background)
cm = partial(colormap_select, reverse=(background=="black"))

def create_image(x_range, y_range, w=plot_width, h=plot_height):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    agg = cvs.points(df, 'dropoff_longitude', 'dropoff_latitude', ds.count('passenger_count'))
    img = tr_fns.shade(agg, cmap=Hot, how='eq_hist')
    return tr_fns.dynspread(img, threshold=0.5, max_px=4)

p = base_plot(background_fill_color=background)
export(create_image(*NYC), "NYCT_hot")
InteractiveImage(p, create_image)

In [ ]:
from functools import partial

def create_image90(x_range, y_range, w=plot_width, h=plot_height):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    agg = cvs.points(df, 'dropoff_longitude', 'dropoff_latitude', ds.count('passenger_count'))
    img = tr_fns.shade(agg.where(agg > np.percentile(agg, 90)), cmap=inferno, how='eq_hist')
    return tr_fns.dynspread(img, threshold=0.3, max_px=4)
    
p = base_plot()
export(create_image(*NYC), "NYCT_90th")
InteractiveImage(p, create_image90)

In [ ]:
def merged_images(x_range, y_range, w=plot_width, h=plot_height, how='log'):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    picks = cvs.points(df, 'pickup_longitude', 'pickup_latitude', ds.count('passenger_count'))
    drops = cvs.points(df, 'dropoff_longitude', 'dropoff_latitude', ds.count('passenger_count'))
    more_drops = tr_fns.shade(drops.where(drops > picks), cmap=["darkblue", 'cornflowerblue'], how=how)
    more_picks = tr_fns.shade(drops.where(picks > drops), cmap=["darkred", 'orangered'], how=how)
    img = tr_fns.stack(more_picks, more_drops)
    return tr_fns.dynspread(img, threshold=0.3, max_px=4)

p = base_plot(background_fill_color=background)
export(merged_images(*NYC), "NYCT_pickups_vs_drops")
InteractiveImage(p, merged_images)

**Reference:**

 - I used Peter Wang's talk "Interactive Viz of a Billion Points with Bokeh Datashader" at PLOTCON 2016 (https://www.youtube.com/watch?v=fB3cUrwxMVY)